In [1]:
import torch
import numpy as np

In [2]:
from torch import nn, optim as opts
from plasma.training import Trainer, metrics, callbacks, data
from plasma.modules import *
from tensorflow.keras.datasets import mnist

In [3]:
(x, y), _ = mnist.load_data()

x = x[y < 3] / 127.5 - 1
y = y[y < 3]

class Data(data.StandardDataset):
    
    def get_len(self):
        return x.shape[0]
    
    def get_item(self, idx):
        return x[idx, None], y[idx]

In [4]:
model = nn.Sequential(*[
    nn.Conv2d(1, 32, kernel_size=7, stride=2, padding=3),
    nn.ReLU(inplace=True),
    nn.Conv2d(32, 64, kernel_size=7, stride=2, padding=3),
    nn.ReLU(inplace=True),
    GlobalAverage(),
    nn.Linear(64, 3)
])

model.cuda(0)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): ReLU(inplace=True)
  (2): Conv2d(32, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (3): ReLU(inplace=True)
  (4): GlobalAverage(axes=[2, 3], keepdims=False)
  (5): Linear(in_features=64, out_features=3, bias=True)
)

In [5]:
loss = nn.CrossEntropyLoss()

In [6]:
#opt = opts.RMSprop(model.parameters())
opt = opts.SGD(model.parameters(), lr=0.25, momentum=0.9, nesterov=True)
trainer = Trainer(model, opt, loss, metrics=[metrics.accuracy], x_device="cuda:0", y_device="cuda:0")

In [7]:
cbs = [
    #callbacks.LrFinder(min_lr=1e-5, max_lr=2, epochs=3)
    callbacks.WarmRestart(1e-5, 10, factor=2, periods=3, snapshot=False),
    #callbacks.CLR(1e-5, 4),
    #callbacks.TrainingScheduler(epochs=1)
    callbacks.CSVLogger("train.csv")
]

In [8]:
trainer.fit(Data(), callbacks=cbs, batch_size=64)

epoch 1



epoch 2



epoch 3



epoch 4



epoch 5


...


epoch 66



epoch 67



epoch 68



epoch 69



epoch 70


In [9]:
cbs[0].plot_data(target="accuracy")

AttributeError: 'WarmRestart' object has no attribute 'plot_data'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

a = Data()[2][0]
ar = np.rot90(a, axes=[1, 2])
ar = np.copy(ar)

_, ax = plt.subplots(ncols=2)
ax[0].imshow(a[0])
ax[1].imshow(ar[0])
plt.show()
a.shape, a.max(), a.min()

In [ ]:
img = torch.tensor(a[None], dtype=torch.float, device="cuda:0")
model.eval()(img)

In [ ]:
img = torch.tensor(ar[None], dtype=torch.float, device="cuda:0")
model.eval()(img)

In [ ]:
img = torch.tensor(ar30[None, None], dtype=torch.float, device="cuda:0")
model.eval()(img)